### 練習問題2.77
Louis Reasoner は、式 (magnitude z) を評価しようとした。<br/>
ここで、z は 図 2.24に⽰したオブジェクトである。<br/>
驚いたことに、5 という答えが返ってくるのではなく、<br/>
apply-generic からのエラーメッセージが返ってきて、型 (complex) には magnitude という演算を⾏う⼿続きがないという。<br/>
彼がこの対話を Alyssa P. Hacker に⾒せると、<br/>
Alyssa は <br/>
“複素数セレクタが polar と rectangular の数値にだけしか定義されていなくて、complex の数値に対して定義されていないのが問題なのよ。<br/>
動くようにするには complex パッケージに以下の式を追加すれば⼤丈夫” <br/>
と⾔う。

    (put 'real-part '(complex) real-part)
    (put 'imag-part '(complex) imag-part)
    (put 'magnitude '(complex) magnitude)
    (put 'angle '(complex) angle)

なぜこれが動くのか、詳しく説明せよ。<br/>
例として、z が図 2.24に⽰したオブジェクトであるとき、式 (magnitude z) を評価した際に呼ばれるすべての⼿続きをトレースせよ。<br/>
具体的には、apply-generic は何回起動されるだろうか。<br/>
それぞれの場合について、ディスパッチ先の⼿続きは何になるだろうか。


In [20]:
(define (lookup key table)
  (let ((record (assoc key (cdr table))))
    (if record (cdr record)
        #f)
    )
  )

(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (insert! key value table)
  (let ((record (assoc key (cdr table))))
    (if record (set-cdr! record value)
        (set-cdr! table (cons (cons key value) (cdr table)))
        )
    )
  'ok)

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

;(define (make-table) (list '*table*))

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))
; タグつきデータのコンストラクタ
(define (attach-tag type-tag contents)
  (cons type-tag contents))
; タグを返すセレクタ
(define (type-tag datum)
  (if (pair? datum) (car datum)
      (error "Bad tagged datum: TYPE-TAG" datum))
  )
; データを返すセレクタ
(define (contents datum)
  (if (pair? datum) (cdr datum)
      (error "Bad tagged datum: CONTENTS" datum))
  )

;
(define (apply-generic op . args)
  (display "apply-generic " op " " args)
  (display "\n")
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
           "No method for these types: APPLY-GENERIC"
           (list op type-tags )))))
  )

In [21]:
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))


In [22]:
(import "math")
(define (square x)(* x x))

(define (install-rectangular-package)
  ;; 内部⼿続き
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y) (cons x y))
  (define (magnitude z) (sqrt (+ (square (real-part z)) (square (imag-part z)))))
  (define (angle z) (math.atan2 (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a) (cons (* r (math.cos a)) (* r (math.sin a))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular (lambda (r a) (tag (make-from-mag-ang r a))))
'done)

(define (install-polar-package)
  ;; 内部⼿続き
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z) (* (magnitude z) (math.cos (angle z))))
  (define (imag-part z) (* (magnitude z) (math.sin (angle z))))
  (define (make-from-real-imag x y) (cons (sqrt (+ (square x) (square y))) (math.atan2 y x)))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

(define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
(define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))

(install-rectangular-package)
(install-polar-package)

done

In [23]:
(define (install-complex-package)
  ;; 直交形式パッケージと極形式パッケージからインポートした⼿続き
  (define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
  (define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))
  ;; 内部⼿続き
  (define (add-complex z1 z2) (make-from-real-imag (+ (real-part z1) (real-part z2)) (+ (imag-part z1) (imag-part z2))))
  (define (sub-complex z1 z2) (make-from-real-imag (- (real-part z1) (real-part z2)) (- (imag-part z1) (imag-part z2))))
  (define (mul-complex z1 z2) (make-from-mag-ang (* (magnitude z1) (magnitude z2)) (+ (angle z1) (angle z2))))
  (define (div-complex z1 z2) (make-from-mag-ang (/ (magnitude z1) (magnitude z2)) (- (angle z1) (angle z2))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag z) (attach-tag 'complex z))
  (put 'add '(complex complex) (lambda (z1 z2) (tag (add-complex z1 z2))))
  (put 'sub '(complex complex) (lambda (z1 z2) (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex) (lambda (z1 z2) (tag (mul-complex z1 z2))))
  (put 'div '(complex complex) (lambda (z1 z2) (tag (div-complex z1 z2))))
  (put 'make-from-real-imag 'complex (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (make-complex-from-real-imag x y) ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a) ((get 'make-from-mag-ang 'complex) r a))

(install-complex-package)

done

In [24]:
(define a (make-complex-from-real-imag 1 2))
(define b (make-complex-from-real-imag 1 3))

(display (add a b))
(display "\n")
(display (sub a b))
(display "\n")
(display (mul a b))
(display "\n")
(display (div a b))
(display "\n")


apply-generic 
apply-generic 
apply-generic 
apply-generic 
apply-generic 
(complex rectangular 2 . 5)
apply-generic 
apply-generic 
apply-generic 
apply-generic 
apply-generic 
(complex rectangular 0 . -1)
apply-generic 
apply-generic 
apply-generic 
apply-generic 
apply-generic 
(complex polar 7.071067811865476 . 2.356194490192345)
apply-generic 
apply-generic 
apply-generic 
apply-generic 
apply-generic 
(complex polar 0.7071067811865475 . -0.14189705460416402)


In [25]:
(define a (make-complex-from-real-imag 3 4))

(magnitude a)


apply-generic 



Traceback (most recent call last):
  File "In [25]", line 3, col 1, in 'magnitude'
  File "In [22]", line 42, col 23, in 'apply-generic'
  File "In [20]", line 102, col 3, in 'let'
  File "In [20]", line 103, col 5, in 'let'
  File "In [20]", line 106, col 11, in 'error'
UnhandledException: string argument expected, got 'Symbol'



In [26]:
(define (install-complex-package)
  ;; 直交形式パッケージと極形式パッケージからインポートした⼿続き
  (define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
  (define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))
  ;; 内部⼿続き
  (define (add-complex z1 z2) (make-from-real-imag (+ (real-part z1) (real-part z2)) (+ (imag-part z1) (imag-part z2))))
  (define (sub-complex z1 z2) (make-from-real-imag (- (real-part z1) (real-part z2)) (- (imag-part z1) (imag-part z2))))
  (define (mul-complex z1 z2) (make-from-mag-ang (* (magnitude z1) (magnitude z2)) (+ (angle z1) (angle z2))))
  (define (div-complex z1 z2) (make-from-mag-ang (/ (magnitude z1) (magnitude z2)) (- (angle z1) (angle z2))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag z) (attach-tag 'complex z))
  (put 'add '(complex complex) (lambda (z1 z2) (tag (add-complex z1 z2))))
  (put 'sub '(complex complex) (lambda (z1 z2) (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex) (lambda (z1 z2) (tag (mul-complex z1 z2))))
  (put 'div '(complex complex) (lambda (z1 z2) (tag (div-complex z1 z2))))
  (put 'make-from-real-imag 'complex (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex (lambda (r a) (tag (make-from-mag-ang r a))))
  ;; 追加
  (put 'real-part '(complex) real-part)
  (put 'imag-part '(complex) imag-part)
  (put 'magnitude '(complex) magnitude)
  (put 'angle '(complex) angle)

  'done)

(define (make-complex-from-real-imag x y) ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a) ((get 'make-from-mag-ang 'complex) r a))

(install-complex-package)

done

In [27]:
(define a (make-complex-from-real-imag 3 4))

(display (magnitude a))
(display "\n")
(display (angle a))
(display "\n")


apply-generic 
apply-generic 
5.0
apply-generic 
apply-generic 
0.9272952180016122


In [28]:
(define b (make-complex-from-real-imag 1 1))

(display (magnitude b))
(display "\n")
(display (angle b))
(display "\n")


apply-generic 
apply-generic 
1.4142135623730951
apply-generic 
apply-generic 
0.7853981633974483


In [19]:
(magnitude b)
->
(apply-generic 'magnitude b)
->
complex <- tape-tag b
pro1c <- complexのmagunitude
data1 <- contents b
->
(magunitude data1)
->
(apply-generic 'rectangular b)
->
rectangular <- tape-tag data1
proc <- rectangularのmagunitude
data2(real imag) <- contents data1


apply-generic apply-generic 


Traceback (most recent call last):
  File "In [19]", line 1, col 1, in 'magnitude'
  File "In [3]", line 42, col 23, in 'apply-generic'
  File "In [14]", line 102, col 3, in 'let'
  File "In [14]", line 103, col 5, in 'let'
  File "In [14]", line 105, col 11, in 'apply'
  File "In [3]", line 42, col 23, in 'apply-generic'
  File "In [14]", line 102, col 3, in 'let'
  File "In [14]", line 103, col 5, in 'let'
  File "In [14]", line 106, col 11, in 'error'
UnhandledException: string argument expected, got 'Symbol'

